# Cortex Game Python Demo-Round 2_Amount

Import SASpy and connect to your SAS studio session


(Instruction of SASpy install and configuaration:
https://support.sas.com/ondemand/saspy.html )

In [1]:
from saspy import SASsession
sas_session = SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 2061



Load raw datasets from SAS Studio (Change the folder path to where your "Cortex Data Sets" folder located.)

In [2]:
%%SAS sas_session

libname cortex '/home/u58717790/my_shared_file_links/u39842936/Cortex Data Sets';

# Step1-2 Merge and DataPartition
link for reference: 

Pandas library: https://pandas.pydata.org/docs/user_guide/index.html


sklearn.model_selection: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [3]:
import pandas as pd

#comment: Transform cloud sas dataset to python datafrmae(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

In [4]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]

#Deal with Missing Value 
data_merge = data_merge.dropna()  #comment: you could use another method to deal with NA
data_merge.head()

#Step2 Data Partition 
#this is just a sample, you could use another library or bulit in funciton
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4) # you can change the percentage
train.head()

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
24355,2024356.0,WILDES,MARIE,1.0,60.0,200900.0,University / College,Suburban,5.0,1.0,...,3.0,5.0,140.0,15.0,75.0,0.0,0.0,0.0,1.0,700.0
954024,2954025.0,LASHINSKI,JOANNE,1.0,50.0,24400.0,High School,Downtown,6.0,3.0,...,3.0,5.0,65.0,20.0,25.0,1.0,10.0,0.0,1.0,100.0
838793,2838794.0,HACHA,YVONNE,1.0,49.0,36800.0,University / College,Suburban,7.0,3.0,...,3.0,6.0,355.0,15.0,300.0,0.0,0.0,0.0,1.0,50.0
469572,2469573.0,GALLEGOS,THOMAS,0.0,32.0,129100.0,University / College,Suburban,5.0,2.0,...,1.0,3.0,30.0,30.0,30.0,0.0,0.0,1.0,1.0,75.0
623780,2623781.0,KEY,ETHEL,1.0,31.0,142100.0,University / College,City,8.0,2.0,...,2.0,4.0,1020.0,20.0,1000.0,1.0,125.0,0.0,1.0,20.0


# Step3 Linear Regression Model (Py)

link for reference:

sk-learn library: https://scikit-learn.org/stable/index.html

In [5]:
from sklearn import linear_model

X_train = train[['Age','Salary','Seniority','GaveLastYear','Contact']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age','Salary','Seniority','GaveLastYear','Contact']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()
regr.fit(X_train,Y_train)
regr_predict=regr.predict(X_valid)

In [6]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

68.3847315473563
53546.91335851458
231.40205997033513


# Step4 Decision Tree Model（Py）

In [7]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age','Salary','Seniority','GaveLastYear','Contact']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age','Salary','Seniority','GaveLastYear','Contact']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 57.0890411  123.03098291  64.60375213 ...  42.12711069  47.00555881
  26.7816092 ]


In [8]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

68.3980750647108
56678.52328075901
238.0725168530778


### Other models may also be helpful for this game:
Link for reference:
1. Logistic regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression
2. Neural Net: https://scikit-learn.org/stable/modules/classes.html?highlight=neural%20net#module-sklearn.neural_network  

# Step6 Scoring the data

Pick the best model from previous steps and use it to predict for next year donation.

In [9]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='SCORE_RD2_NOCONTACT',
libref='cortex'
)

In [10]:
#Predict amount give given contact

scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")
scoring_data_contact = scoring_data_contact.dropna() 
scoring_data_contact.head()

X = scoring_data_contact[['Age','Salary','Seniority','GaveLastYear','Contact']] 
regr_predict_contact=regr.predict(X)
scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,89.953853
9,2000010.0,79.008773
14,2000015.0,85.867961
15,2000016.0,88.201553
16,2000017.0,48.970617


In [11]:
#Predict amount give given not contact

scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")
scoring_data_nocontact = scoring_data_nocontact.dropna() 
scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age','Salary','Seniority','GaveLastYear','Contact']] 
regr_predict_nocontact=regr.predict(X)
scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,85.342560
9,2000010.0,74.397481
14,2000015.0,81.256668
15,2000016.0,83.590261
16,2000017.0,44.359325


In [12]:
result = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result.sort_values(by=['ID'], inplace=True)
result.head()

,ID,AmtContact,AmtNoContact
0,2000001.0,89.953853,85.342560
1,2000010.0,79.008773,74.397481
2,2000015.0,85.867961,81.256668
3,2000016.0,88.201553,83.590261
4,2000017.0,48.970617,44.359325


# Step7 Exporting CSV (Py)

In [15]:
result.to_csv('Round2_Output_amt.csv', index=False)